20 февраля, Пятерочка изменила структуру сайта чуть-чуть 😈

В этом блокноте мы занимаемся скачиванием продуктов с сайта Пятерочки

Для начала нам нужно импортировать нужные библиотеки и задать константы. Это параметры запросов к сайту.

In [41]:
import requests
import json
import time
import random
import pandas as pd
from datetime import date

# define constants for requests

BASE_URL = "https://5d.5ka.ru/api/catalog/v2/stores/Y233/categories/"
CATEGORIES_URL = "https://5d.5ka.ru/api/catalog/v2/stores/Y233/categories?mode=delivery&include_subcategories=1"
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
CHECK_DATE = date.today() # current date for reference

HEADERS = {
    "user-agent": USER_AGENT,
    "origin": "https://5ka.ru"
}

Так как скачивать продукты мы будем по категориям, нужно получить список категорий товаров с сайта. За это будет отвечать следующая функция, которая запросит у сайта список категорий и подкатегорий, выберет только id и названия категорий и сохранит данные в словарь.

In [42]:
def fetch_categories():
    
    response = requests.get(CATEGORIES_URL, headers=HEADERS) # get the data
    raw_categories = json.loads(response.text) # convert into a dictionary
    
    cleaned_categories = []
    
    # go through the raw data and select only the necessary fields
    for category in raw_categories:    # select the broad category, save its id and name
        for subcategory in category["categories"]:    # go through all subcategories and save them, specify parent_category
            cleaned_categories.append({
                "id": subcategory["id"],
                "name": subcategory["name"],
                "parent_id": category["id"]
            })

    return cleaned_categories

Загружаем категории в отдельном блоке, чтобы не повторять эту операцию. ***Временно также выбираем подмножество категорий, чтобы не скачивать сразу все.

In [47]:
categories = fetch_categories()
categories = categories[83:]
total_categories_count = len(categories)
categories_df = pd.DataFrame(categories)
# categories_df.to_csv(f'categories-{CHECK_DATE}.csv', index=False)
print(categories)

[{'id': '251C13033', 'name': 'Детские напитки', 'parent_id': '251C12906'}, {'id': '251C13034', 'name': 'Детская гигиена', 'parent_id': '251C12906'}, {'id': '251C13035', 'name': 'Игры и игрушки', 'parent_id': '251C12906'}, {'id': '251C13036', 'name': 'Школа и хобби', 'parent_id': '251C12906'}, {'id': '251C13029', 'name': 'Для кошек', 'parent_id': '251C12907'}, {'id': '251C13030', 'name': 'Для собак', 'parent_id': '251C12907'}, {'id': '251C13023', 'name': 'Уход за волосами', 'parent_id': '251C12908'}, {'id': '251C13024', 'name': 'Уход за телом и лицом', 'parent_id': '251C12908'}, {'id': '251C13025', 'name': 'Уход за полостью рта', 'parent_id': '251C12908'}, {'id': '251C13026', 'name': 'Бумага и салфетки', 'parent_id': '251C12908'}, {'id': '251C13027', 'name': 'Аптека', 'parent_id': '251C12908'}, {'id': '251C13028', 'name': "Л'Этуаль в Пятёрочке", 'parent_id': '251C12908'}, {'id': '251C13018', 'name': 'Стирка', 'parent_id': '251C12909'}, {'id': '251C13019', 'name': 'Для мытья посуды', 'pa

Собственно код при поддержке ЧатГПТ поделен на осмысленные функции. Следующая функция делает запрос к сайту для получения данных о продуктах в указанной категории. Данные возвращаются как JSON в том формате, в котором они существуют на сайте, функция выбирает из них только данные о продуктах и сохраняет как словарь.

In [44]:
# fetch the raw product data (the site uses an increasing limit to fetch all the products, we mimic this behavior)

def fetch_products(category_id):
    
    current_limit = 0
    total_products = 0
    products = []
    
    while total_products >= current_limit: # loop until the number of products gets less than limit
        time.sleep(random.uniform(1, 5)) # random time delay to avoid being blocked
        
        current_limit += 12 # set initial limit as 30 and increment it with each iteration
        url = f'{BASE_URL}{category_id}/products?mode=delivery&include_restrict=false&limit={current_limit}'
        
        response = requests.get(url, headers=HEADERS) # get the data
        response_data = json.loads(response.text) # convert into a dictionary
        products = response_data["products"] # extract only products
        total_products = len(products) # check the total number of products
        print(f'{len(products)}..', end='') # progress indication

    return products
    

Следующая функция берет необработанные данные в виде словаря и создает DataFrame, в который отбираются только нужные мне параметры товара: название, цена, единица измерения. Пока примерно)

In [45]:
def clean_product_data(category, raw_products):

    cleaned_products = []

    # go through the raw data and select only the necessary fields
    for product in raw_products:
        cleaned_products.append({
            "category_id": category,
            "name": product["name"],
            "unit_of_measurement": product["uom"],
            "price_reg": product["prices"]["regular"],
            "price_disc": product["prices"]["discount"],
            "pricing_clarification": product["property_clarification"] # some extra parameter, apparently it clarifies the unit for the price or the net weight
        })

    return cleaned_products

Наконец, в следующей ячейке находится основное тело программы, где мы вызываем эти функции. Сначала запрашиваем список категорий, в цикле скачиваем данные о продуктах, сохраняем все в один большой DataFrame с указанием даты запроса.

In [48]:
fetched_categories_count = 0    # counter for fetching progress tracker

# Create the file first with headers
products_df = pd.DataFrame(columns=["category_id", "name", "unit_of_measurement", "price_reg", "price_disc", "pricing_clarification"])
products_df.to_csv(f'products-{CHECK_DATE}.csv', index=False, mode='w')

for category in categories:
    raw_products = fetch_products(category["id"]) # fetch products
    new_products = clean_product_data(category["id"], raw_products) # select only relevant data and add new products to the list

    products_df = pd.DataFrame(new_products)
    products_df.to_csv(f'products-{CHECK_DATE}.csv', index=False, mode='a', header=False)

    fetched_categories_count += 1
    print(f'{category["id"]} finished, {fetched_categories_count} out of {total_categories_count} categories fetched')
    time.sleep(random.uniform(1, 5))

print(f'Fetching complete, see the result in products-{CHECK_DATE}.csv')

12..24..36..48..60..68..251C13033 finished, 1 out of 22 categories fetched
12..24..36..48..60..62..251C13034 finished, 2 out of 22 categories fetched
12..24..36..48..60..72..84..96..108..112..251C13035 finished, 3 out of 22 categories fetched
12..24..36..37..251C13036 finished, 4 out of 22 categories fetched
12..24..36..48..60..72..84..96..108..120..121..251C13029 finished, 5 out of 22 categories fetched
12..24..31..251C13030 finished, 6 out of 22 categories fetched
12..24..36..48..60..72..81..251C13023 finished, 7 out of 22 categories fetched
12..24..36..48..60..72..84..96..108..120..132..144..156..168..180..192..204..216..228..240..252..252..251C13024 finished, 8 out of 22 categories fetched
12..24..36..48..48..251C13025 finished, 9 out of 22 categories fetched
12..24..36..48..56..251C13026 finished, 10 out of 22 categories fetched
12..24..36..48..48..251C13027 finished, 11 out of 22 categories fetched
12..24..36..48..60..72..78..251C13028 finished, 12 out of 22 categories fetched
12

---

All of the above is dedicated to scraping

All of the below is dedicated to cleaning (for now to avoid errors it's better not to execute cells above, except the libraries)

---

In [1]:
import pandas as pd
import re

In [2]:
categories = pd.read_csv('categories-2025-03-03.csv')
products_original = pd.read_csv('products-2025-03-03-complete.csv')
products = products_original.drop(['category_id', 'price_disc', 'unit_of_measurement'], axis=1)    # drop category_id, uom and price_disc columns as they're actually unnecessary
products = products.rename(columns={'price_reg': 'price', 'pricing_clarification': 'pricing_unit'})    # rename price column for simplicity

In [51]:
categories

,id,name,parent_id
0,251C14820,Комбо Экономбо,251C12884
1,251C12891,Завтраки,251C12884
2,251C13103,Перекусы и напитки,251C12884
3,251C13979,Салаты и закуски,251C12884
4,251C13980,Основные блюда и супы,251C12884
...,...,...,...
94,251C12942,Кухня,251C12910
95,251C12943,Дача и отдых,251C12910
96,251C12944,Хозтовары,251C12910
97,251C12945,Декор для дома,251C12910


In [52]:
products

,name,price,pricing_unit
0,Салат сельдь под шубой Пятерочка Кафе 250г,179.99,250 г
1,Салат с кальмаром Пятерочка Кафе 200г,189.99,200 г
2,Салат Оливье Пятерочка Кафе 200г,139.99,200 г
3,Салат Крабовый Пятерочка Кафе 200г,159.99,200 г
4,Салат Гнездо глухаря Пятерочка Кафе 200г,189.99,200 г
...,...,...,...
7149,Колготки Omsa 40den черные размер XL 1пара,308.99,1 шт
7150,Колготки Omsa 40 Nero размер 3-М,308.99,1 шт
7151,Колготки Conte Elegant Slim Silhouette Contour...,257.99,1 шт
7152,Колготки Omsa 40 den черные размер 4-L,308.99,1 шт


In [3]:
products = products.drop_duplicates() # remove duplicates

Let's practice locating rice

In [50]:
rice = products.loc[products.name.str.contains(r'^рис ', case=False, regex=True)]
rice

,name,price,pricing_unit
3155,Рис Global Village Для плова 800г,124.99,800 г
3156,Рис Селяночка круглозерный 5x80г,106.99,400 г
3157,Рис круглозернистый шлифованный 900г,49.99,900 г
3160,Рис Селяночка длиннозерный пропаренный 900г,139.99,900 г
3162,"Рис Селяночка круглозёрный шлифованный, 900г",129.99,900 г
3173,Рис Увелка круглозерный шлифованный 500г,84.99,500 г
3177,Рис пропаренный шлифованный 900г,112.99,900 г
3180,Рис Селяночка длиннозерный первый сорт 5х80г,124.99,400 г
3183,Рис Global Village Басмати шлифованный 450г,159.99,450 г
3188,Рис Мистраль Янтарь длиннозерный пропаренный 5...,129.99,400 г


Next piece of code is extracting weight info from the names (with regex) and adds two new columns to the dataframe: weight, and price per kg.

In [51]:
def extract_weight(name):
    # if there're small portions
    match = re.search(r'(\d+)(x|х)(\d+)\s?г', name)
    if match:
        portion, per_portion = map(int, match.group(1,3))
        return portion * per_portion
    # if there's a single weight
    match = re.search(r'(\d+)\s?г', name)
    if match:
        weight = int(match.group(1))
        return weight

rice = rice.copy()  # recreate the dataframe
rice.loc[:,'weight'] = rice.name.apply(extract_weight)  # a column with weigths in grams
rice.loc[:,'price_kg'] = rice.price / rice.weight * 1000   # a column with prices per kg
# let's also add columns that would be helpful later: product_type and supermarket
rice.loc[:,'product_type'] = 'Rice'
rice.loc[:,'supermarket'] = 'Pyaterochka'
rice

,name,price,pricing_unit,weight,price_kg,product_type,supermarket
3155,Рис Global Village Для плова 800г,124.99,800 г,800,156.237500,Rice,Pyaterochka
3156,Рис Селяночка круглозерный 5x80г,106.99,400 г,400,267.475000,Rice,Pyaterochka
3157,Рис круглозернистый шлифованный 900г,49.99,900 г,900,55.544444,Rice,Pyaterochka
3160,Рис Селяночка длиннозерный пропаренный 900г,139.99,900 г,900,155.544444,Rice,Pyaterochka
3162,"Рис Селяночка круглозёрный шлифованный, 900г",129.99,900 г,900,144.433333,Rice,Pyaterochka
3173,Рис Увелка круглозерный шлифованный 500г,84.99,500 г,500,169.980000,Rice,Pyaterochka
3177,Рис пропаренный шлифованный 900г,112.99,900 г,900,125.544444,Rice,Pyaterochka
3180,Рис Селяночка длиннозерный первый сорт 5х80г,124.99,400 г,400,312.475000,Rice,Pyaterochka
3183,Рис Global Village Басмати шлифованный 450г,159.99,450 г,450,355.533333,Rice,Pyaterochka
3188,Рис Мистраль Янтарь длиннозерный пропаренный 5...,129.99,400 г,400,324.975000,Rice,Pyaterochka


In [52]:
rice.describe()

,price,weight,price_kg
count,27.000000,27.000000,27.000000
mean,146.871481,701.851852,245.685021
std,42.117720,237.568404,157.508868
min,49.990000,250.000000,55.544444
25%,124.990000,425.000000,159.779861
50%,146.990000,800.000000,199.988889
75%,169.990000,900.000000,289.975000
max,224.990000,900.000000,899.960000


Next, I want to generalize rice operations onto more product types
Workflow is as following:
1) Filtering products
2) Extracting units for normalization
3) Normalize units, add them to the dataset
4) *Add a product type and a supermarket name

To generalize step 1, we need a mapping (a dictionary) of regex for each product type

In [90]:
product_regex_map = {
    'rice': r'(^|^")рис\b',
    'bread': r'(^хлеб\b|^багет\b|^батон\b)(?!.*(чесн|заморож))',
    'chicken_fillet': r'^филе\b.*(кур|цыпл)(?!.*запеч)',
    'pork_leg': r'^окорок\b.*свин',
    'egg': r'^яйцо.*курин',
    'cucumber': r'^огур(цы|ец)(?!.*(солен|маринован))',
    'carrot': r'^морковь(?!.*корей)',
    'onion': r'^лук.*реп(?!.*зелен)',
    'tomato': r'^томаты(?!.*(сок|очищ|маринован|вялен|солен))',
    'cabbage': r'^капуста\b.*белокоч',
    'eggplant': r'^баклажаны?($|.*теплич)',
    'banana': r'^банан',
    'orange': r'^апельсин',
    'milk': r'^молоко(?!.*(сгущ|сух))',
    'yogurt': r'^йогурт\b(?!.*питье)',
    'condensed_milk': r'(^молоко.*сгущ|^сгущ)(?!.*(варен|какао|шокол))',
    'green_tea': r'^чай.* зел(?!.*(порош|л$))',
    'black_tea': r'^чай.* черн(?!.*л$)',
    'ground_coffee': r'^кофе(?!.*(капсул|раствор)).*молот',
    'sugar': r'^сахар\b(?!.*ванил)',
    'salt': r'^соль(?!.*(розов|посуд|чесн|ванн|спец))',
    'sunflower_oil': r'^масло\b.*подсолн(?!.*добавл)',
    'water': r'^вода(?!.*(малин|лимон)).*негаз',
    'buckwheat': r'(^крупа\b.*гречн|^гречка\b)(?!.*(\bпшен|\bкиноа))',
    'spaghetti': r'(^макароны.*спагетти|^спагетти\b)(?!.*(заморож|кукуруз))',
    'rice_noodles': r'(^лапша|^вермишель).*(фунчоз)',
    'tofu': r'^тофу\b',
    'mango': r'^манго\b(?!.*(суш|заморож))'
}
product_regex_list = '|'.join(product_regex_map.values())
product_regex_list

'(^|^")рис\\b|(^хлеб\\b|^багет\\b|^батон\\b)(?!.*(чесн|заморож))|^филе\\b.*(кур|цыпл)(?!.*запеч)|^окорок\\b.*свин|^яйцо.*курин|^огур(цы|ец)(?!.*(солен|маринован))|^морковь(?!.*корей)|^лук.*реп(?!.*зелен)|^томаты(?!.*(сок|очищ|маринован|вялен|солен))|^капуста\\b.*белокоч|^баклажаны?($|.*теплич)|^банан|^апельсин|^молоко(?!.*(сгущ|сух))|^йогурт\\b(?!.*питье)|(^молоко.*сгущ|^сгущ)(?!.*(варен|какао|шокол))|^чай.* зел(?!.*(порош|л$))|^чай.* черн(?!.*л$)|^кофе(?!.*(капсул|раствор)).*молот|^сахар\\b(?!.*ванил)|^соль(?!.*(розов|посуд|чесн|ванн|спец))|^масло\\b.*подсолн(?!.*добавл)|^вода(?!.*(малин|лимон)).*негаз|(^крупа\\b.*гречн|^гречка\\b)(?!.*(\\bпшен|\\bкиноа))|(^макароны.*спагетти|^спагетти\\b)(?!.*(заморож|кукуруз))|(^лапша|^вермишель).*(фунчоз)|^тофу\\b|^манго\\b(?!.*(суш|заморож))'

In [91]:
filtered_products = products.loc[products.name.str.contains(product_regex_list, case=False, regex=True)]
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(filtered_products)

C:\Users\Just Alex\AppData\Local\Temp\ipykernel_1032\2593560901.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_products = products.loc[products.name.str.contains(product_regex_list, case=False, regex=True)]


,name,price,pricing_unit
293,Багет традиционный 230г,45.99,230 г
302,Багет Фитнес 200г,49.99,200 г
312,Багет мини 120г,25.99,120 г
320,Хлеб Маг с семечками 390г,75.99,390 г
608,Огурцы среднеплодные,175.99,Цена за 1 кг
609,Томаты,203.99,Цена за 1 кг
610,Капуста белокочанная,39.99,Цена за 1 кг
612,Лук репчатый,59.99,Цена за 1 кг
613,Огурцы короткоплодные 450г,259.99,450 г
614,Огурец Global Village Агромос длинный 1шт.,118.99,1 шт


Let's add missing tags for product type and supermarket, for that we have to match products to their types by matching regex again

In [92]:
def assign_product_type(row):
    name = row['name']
    for product_type, regex in product_regex_map.items():
        match = re.search(regex, name, flags=re.IGNORECASE)
        if match:
            return product_type
    return None

filtered_products = filtered_products.copy()  # recreate the dataframe
filtered_products.loc[:,'product_type'] = filtered_products.apply(assign_product_type, axis=1)
filtered_products.loc[:,'supermarket'] = 'Pyaterochka'
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(filtered_products)

,name,price,pricing_unit,product_type,supermarket
293,Багет традиционный 230г,45.99,230 г,bread,Pyaterochka
302,Багет Фитнес 200г,49.99,200 г,bread,Pyaterochka
312,Багет мини 120г,25.99,120 г,bread,Pyaterochka
320,Хлеб Маг с семечками 390г,75.99,390 г,bread,Pyaterochka
608,Огурцы среднеплодные,175.99,Цена за 1 кг,cucumber,Pyaterochka
609,Томаты,203.99,Цена за 1 кг,tomato,Pyaterochka
610,Капуста белокочанная,39.99,Цена за 1 кг,cabbage,Pyaterochka
612,Лук репчатый,59.99,Цена за 1 кг,onion,Pyaterochka
613,Огурцы короткоплодные 450г,259.99,450 г,cucumber,Pyaterochka
614,Огурец Global Village Агромос длинный 1шт.,118.99,1 шт,cucumber,Pyaterochka


It's going slowly, now we have 5 products, let's try to deal with them. Steps 2 are extracting weight or number, calculating normalized price and adding missing tags

In [93]:
def extract_weight(row):
    name, pricing_unit = row['name'], row['pricing_unit']

    # first, check name
    # calculate weight if there're multiple portions
    match = re.search(r'(\d+)(x|х)(\d+|\d+[.]\d+)\s?г', name)
    if match:
        portion, per_portion = map(float, match.group(1,3))
        return portion * per_portion
    # if there's a single weight
    match = re.search(r'(\d+|\d+[.]\d+)\s?(г|кг)', name)
    if match:
        weight = float(match.group(1))
        unit = match.group(2)
        return weight * 1000 if unit == 'кг' else weight
    # if name doesn't contain anything, check pricing_unit
    match = re.search(r'(\d+|\d+[.]\d+)\s?(г|кг)', pricing_unit)
    if match:
        weight = float(match.group(1))
        unit = match.group(2)
        return weight * 1000 if unit == 'кг' else weight

    return None  # if nothing matched

def extract_number_of_units(row):
    name, pricing_unit = row['name'], row['pricing_unit']

    # first, check name
    match = re.search(r'(\d+)\s?шт', name)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units
    # if name doesn't contain anything, check pricing_unit
    match = re.search(r'(\d+)\s?шт', pricing_unit)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units

    return None  # if nothing matched

def extract_volume(row):
    name, pricing_unit = row['name'], row['pricing_unit']

    # first, check name
    # calculate volume if there're multiple portions
    match = re.search(r'(\d+)(x|х)(\d+|\d+[.]\d+)\s?мл', name)
    if match:
        portion, per_portion = map(float, match.group(1,3))
        return portion * per_portion
    # if there's a single volume
    match = re.search(r'(\d+|\d+[.]\d+)\s?(мл|л\b)', name)
    if match:
        volume = float(match.group(1))
        unit = match.group(2)
        return volume * 1000 if unit == 'л' else volume
    # if name doesn't contain anything, check pricing_unit
    match = re.search(r'(\d+|\d+[.]\d+)\s?(мл|л\b)', pricing_unit)
    if match:
        volume = float(match.group(1))
        unit = match.group(2)
        return volume * 1000 if unit == 'л' else volume

    return None  # if nothing matched

filtered_products = filtered_products.copy()  # recreate the dataframe
# extract and calculate weights
filtered_products.loc[:,'weight'] = filtered_products.apply(extract_weight, axis=1)  # a column with weigths in grams
filtered_products.loc[:,'price_kg'] = filtered_products.price / filtered_products.weight * 1000   # a column with prices per kg
# extract and calculate number of units (for products with "шт")
filtered_products.loc[:,'number_of_units'] = filtered_products.apply(extract_number_of_units, axis=1)  # a column with number of units
filtered_products.loc[:,'price_unit'] = filtered_products.price / filtered_products.number_of_units   # a column with prices per unit
# extract and calculate volume
filtered_products.loc[:,'volume'] = filtered_products.apply(extract_volume, axis=1)  # a column with volume in ml
filtered_products.loc[:,'price_lit'] = filtered_products.price / filtered_products.volume * 1000   # a column with prices per liter
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(filtered_products)

,name,price,pricing_unit,product_type,supermarket,weight,price_kg,number_of_units,price_unit,volume,price_lit
293,Багет традиционный 230г,45.99,230 г,bread,Pyaterochka,230.0,199.956522,NaN,NaN,NaN,NaN
302,Багет Фитнес 200г,49.99,200 г,bread,Pyaterochka,200.0,249.950000,NaN,NaN,NaN,NaN
312,Багет мини 120г,25.99,120 г,bread,Pyaterochka,120.0,216.583333,NaN,NaN,NaN,NaN
320,Хлеб Маг с семечками 390г,75.99,390 г,bread,Pyaterochka,390.0,194.846154,NaN,NaN,NaN,NaN
608,Огурцы среднеплодные,175.99,Цена за 1 кг,cucumber,Pyaterochka,1000.0,175.990000,NaN,NaN,NaN,NaN
609,Томаты,203.99,Цена за 1 кг,tomato,Pyaterochka,1000.0,203.990000,NaN,NaN,NaN,NaN
610,Капуста белокочанная,39.99,Цена за 1 кг,cabbage,Pyaterochka,1000.0,39.990000,NaN,NaN,NaN,NaN
612,Лук репчатый,59.99,Цена за 1 кг,onion,Pyaterochka,1000.0,59.990000,NaN,NaN,NaN,NaN
613,Огурцы короткоплодные 450г,259.99,450 г,cucumber,Pyaterochka,450.0,577.755556,NaN,NaN,NaN,NaN
614,Огурец Global Village Агромос длинный 1шт.,118.99,1 шт,cucumber,Pyaterochka,NaN,NaN,1.0,118.990000,NaN,NaN


In [95]:
filtered_products.to_csv(f'filtered_products-2025-03-03-pyaterochka.csv')